## Lab 12 Character Sequence RNN

## Character Sequence RNN 1
short sentence

In [ ]:
import tensorflow as tf
import numpy as np
from time import time
tf.set_random_seed(777)  # reproducibility

In [ ]:
sample = " if you want you"
idx2char = list(set(sample))  # 고유한 char 만 추출
char2idx = {c: i for i, c in enumerate(idx2char)}  # char -> idex
print(' idx2char :', idx2char, '\n\n', 'char2idx :', char2idx)

In [ ]:
# hyper parameters
dic_size = len(char2idx)     # 입력값             : RNN input size (one hot size)
hidden_size = len(char2idx)  # 각 cell의 Hidden값 : RNN output size
num_classes = len(char2idx)  # 출력값             : final output size (RNN or softmax, etc.)
batch_size = 1                     # 1번 처리시 1줄씩 입력 : one sample data, one batch
sequence_length = len(sample) - 1  # 1번 처리시 활용할 Cell(단어)의 수 : number of lstm rollings (unit #)
learning_rate = 0.1

In [ ]:
# 데이터 구조형식을 정의
sample_idx = [char2idx[c] for c in sample]  # char to index
x_data = [sample_idx[:-1]]  # X data sample (0 ~ n-1) hello: hell
y_data = [sample_idx[1:]]   # Y label sample (1 ~ n) hello: ello

X = tf.placeholder(tf.int32, [None, sequence_length])  # X data
Y = tf.placeholder(tf.int32, [None, sequence_length])  # Y label

In [ ]:
# RNN Cell 들을 정의
x_one_hot = tf.one_hot(X, num_classes)  # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, x_one_hot, initial_state=initial_state, dtype=tf.float32)

In [ ]:
# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

In [ ]:
# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
prediction = tf.argmax(outputs, axis=2)

In [ ]:
# Creates a graph. ( by CPU )
t1 = time()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        if i % 15 == 0:
            print(i, "loss:", l, "Prediction:", ''.join(result_str))
print( int(time()-t1), 'sec')

In [ ]:
# Creates a graph. ( by GPU )
from time import time
t0 = time()
with tf.device('/gpu:0'):
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        for i in range(100):
            l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
            result = sess.run(prediction, feed_dict={X: x_data})
            # print char using dic
            result_str = [idx2char[c] for c in np.squeeze(result)]
            if i % 15 == 0:
                print(i, "loss:", l, "Prediction:", ''.join(result_str))
print(  int(time()-t0) , 'sec')

In [ ]:
# Creates a graph.
with tf.device('/gpu:0'):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
    c = tf.matmul(a, b)
# Creates a session with log_device_placement set to True.
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
# Runs the op.
print(sess.run(c))